<center><h1>Method 1: Local TFIDF</h1></center> 

- In this method, we perform the localization __only directly__ using the the TFIDF similarity score of bug reports to source codes in each project individually. 
- Note that TF.IDF scores are calculated using the __local data__. That means that for each term we calculate the term frequency and inverse document frequency of that term using only the bug reports and source codes of that project. 
- Refer to the following table to better understand the details of this method.

<img src="Methods.png">

### Libraries

In [ ]:
from __future__ import division
import pandas as pd
import numpy as np

import os
from os import listdir
from os.path import isfile, join
import operator
import traceback
import logging

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from gensim.test.utils import get_tmpfile
from gensim import similarities
from gensim import models

import warnings
import math
import multiprocessing
from tqdm import tqdm_notebook
from time import gmtime, strftime

warnings.simplefilter(action='ignore', category=FutureWarning)

<center><h1>Evaluators</h1></center> 

In [ ]:
def evaluate_helper(ranked_files,fixes):
    """
    @Receives: list of ranked files(which is predicted by algorithm) and the ground truth(fixes)
    @Process: This is a function aimed to help evaluate, and calculates the AP and first_pos 
    @Returns: A dictionary containing the AP(average precision) and first_pos (first retrieved fix file)
    """
    found=0
    first_pos=-1
    average_precision=0
    for i,predictionFix in enumerate(ranked_files):
        for actualFix in fixes:
            if actualFix in predictionFix:
                if first_pos==-1:
                    first_pos=i+1
                found+=1
                average_precision+=found/(i+1)        

    AP=average_precision/found if found>0 else 0
    return {"AP":AP,"first_pos":first_pos}


def evaluate(all_bugs_df,source_codes_df):
    """
    @Receives: The main dataframe
    @Process: Evaluates the predicted files for each bugreport in all_bugs_df
    @Returns: MAP and MRR calculated from eligible bugreports(the ones with
    at least one fix file in this version of code) in the dataframe and number of eligible bug reports.
    """
    all_results=[]
    top_founds=[]
    average_precisions=[]
    for i,br in all_bugs_df.iterrows():
        if not source_codes_df.loc[source_codes_df.filename.apply(lambda filename: any(fix in filename for fix in br['fix']))].empty:
            predicted_files=br['total_score'].keys()
            result=evaluate_helper(predicted_files,br['fix'])
            top_founds.append(result['first_pos'])
            average_precisions.append(result['AP'])
            all_results.append(result)
        else:
            top_founds.append(-1)
            average_precisions.append(0.0)
    all_bugs_df.loc[:,"top_found"]=top_founds
    all_bugs_df.loc[:,"average_precision"]=average_precisions
    
    #Calculating the MAP and MRR
    MAP,MRR=(0,0)
    if len(all_results)>0:
        for result in all_results:
            MAP+=result['AP']
            MRR+=1/result['first_pos'] if result['first_pos']>0 else 0
        MAP/=len(all_results)
        MRR/=len(all_results)
        print("eligible_br_count: ",len(all_results))
    return (MAP,MRR,len(all_results))


<center><h1>TFIDF</h1></center> 

In [ ]:
def getNormValue(x,maximum,minimum):
    return 6*((x - minimum)/(maximum - minimum))

def getLenScore(length):
    return (math.exp(length) / (1 + math.exp(length)))

def calulateLengthScore(source_codes_df):
    """
    Receives: a list of sizes of codes and the index
    Process: calculate a boost score for the specified index based on length of that code
    Returns: length boosting score 
    """
    average_size=source_codes_df['size'].mean()
    standard_deviation=source_codes_df['size'].std() 
    low=average_size-3*standard_deviation
    high= average_size+3*standard_deviation
    minimum=int(low) if low>0 else 0
        
    len_scores=[]
    for i,eachLen in source_codes_df['size'].items():
        score=0
        nor=getNormValue(eachLen,high,minimum)
        if eachLen!=0:
            if eachLen>=low and eachLen<=high:
                score=getLenScore(nor)
            elif eachLen<low:
                score=0.5
            elif eachLen>high:
                score = 1.0
        len_scores.append(score)
    source_codes_df.loc[:,'lengthScore']=len_scores

    return source_codes_df
    
def inverse_doc_freq(idf,D):
    return math.log(D/idf)

def term_freq(tf_list):
    return [(math.log(tf+1)) for tf in tf_list]

def np_normalizer(arr):
    """
    @Receives: a list of numbers
    @Process: normalizes all the values and map them to range of [0,1]
    @Returns: list of normalized numbers
    """
    if len(arr)>0:
        maximum=np.amax(arr)
        minimum=np.amin(arr)
        if maximum!=minimum:
            return (arr-minimum)/(maximum-minimum)
    return arr

def normalizer(Dict):
    """
    @Receives: a list of numbers
    @Process: normalizes all the values and map them to range of [0,1]
    @Returns: list of normalized numbers
    """
    if len(Dict)>0:
        maximum=max(Dict.items(), key=operator.itemgetter(1))[1]
        minimum=min(Dict.items(), key=operator.itemgetter(1))[1]
        for key,value in Dict.items():
            if maximum!=minimum:
                Dict[key]=(value-minimum)/(maximum-minimum)
            else:
                Dict[key]=1.0
    return Dict
    

def TFIDF_transform(all_bugs_df,source_codes_df):

    dictionary = gensim.corpora.Dictionary(list(source_codes_df['code']))
    corpus = [dictionary.doc2bow(doc) for doc in list(source_codes_df['code'])]
    tfidf_weights = models.TfidfModel(corpus,wlocal=term_freq,wglobal=inverse_doc_freq,normalize=False)
    source_codes_df.loc[:,'tfidf_vector']=tfidf_weights[corpus]
    all_bugs_df['tfidf_vector']=all_bugs_df.text.apply(lambda x: tfidf_weights[dictionary.doc2bow(x)])
    return (all_bugs_df,source_codes_df,len(dictionary))

def cos_matrix_multiplication(matrix, vector):
    """
    Calculating pairwise cosine distance using matrix vector multiplication.
    """
    dotted = matrix.dot(vector)
    matrix_norms = np.linalg.norm(matrix, axis=1)
    vector_norm = np.linalg.norm(vector)
    matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
    neighbors = np.divide(dotted, matrix_vector_norms)
    return neighbors


<center><h1>Main BugLocalization class</h1></center> 

In [ ]:
class BugLocalizer:

    all_projects_source_codes=pd.DataFrame([])
    all_projects_bugreports=pd.DataFrame([])
    dataFolder=""
    
    def __init__(self,project,result_path):
        self.project=project
        self.resultPath=result_path
            
    def execute(self):
        print("\t ****** Localizing Bugs for project: {} ******".format(self.project))
        self.loadBugCurpus()
        self.loadSourceFiles()
        self.localize()
        self.evaluate()
        self.to_csv()
        self.write_result()
        
    def loadEverything():
        if BugLocalizer.all_projects_bugreports.empty:
            bugReportFile=os.path.join(BugLocalizer.dataFolder,'allBugReports.pickle')
            if not os.path.isfile(bugReportFile):
                print("The bug reports file (allBugReports.pickle) does not exist. please run the step0 first")
            else: 
                BugLocalizer.all_projects_bugreports=pd.read_pickle(bugReportFile)
                print("*** All Bug Reports are Loaded. ***")

        if BugLocalizer.all_projects_source_codes.empty:
            sourceCodeFile=os.path.join(BugLocalizer.dataFolder,'allSourceCodes.pickle')
            if not os.path.isfile(sourceCodeFile):
                print("The source codes file (allSourceCodes.pickle) does not exist. please run the step0 first")
            else:
                BugLocalizer.all_projects_source_codes=pd.read_pickle(sourceCodeFile)
                print("*** All Source Codes are Loaded. ***")
    
    def loadBugCurpus(self):
        self.all_bugs_df=BugLocalizer.all_projects_bugreports.loc[BugLocalizer.all_projects_bugreports['project']==self.project,:]
        
    def loadSourceFiles(self):
        self.source_codes_df=BugLocalizer.all_projects_source_codes.loc[BugLocalizer.all_projects_source_codes['project']==self.project,:]
    
    def localize(self):
        
        print("Localizing Now ...")
        self.source_codes_df=calulateLengthScore(self.source_codes_df)
        dictionary = gensim.corpora.Dictionary(list(self.source_codes_df['code']))
        corpus = [dictionary.doc2bow(doc) for doc in list(self.source_codes_df['code'])]
        tfidf_weights = models.TfidfModel(corpus,wlocal=term_freq,wglobal=inverse_doc_freq,normalize=False)
        self.source_codes_df.loc[:,'tfidf_vector']=tfidf_weights[corpus]
        self.all_bugs_df.loc[:,'tfidf_vector']=self.all_bugs_df.text.apply(lambda x: tfidf_weights[dictionary.doc2bow(x)])
        scores=[]
        tfidf_index = similarities.SparseMatrixSimilarity(list(self.source_codes_df.tfidf_vector),num_features=len(dictionary))
        for i, br in tqdm_notebook(self.all_bugs_df.iterrows()):
            try:            
                direct_tfidf_similarities=tfidf_index[br.tfidf_vector]
                direct_tfidf_similarities=np_normalizer(direct_tfidf_similarities)
                sourceCodeScores={self.source_codes_df.iloc[j].filename: (direct_tfidf_similarities[j])*self.source_codes_df.iloc[j].lengthScore 
                                                  for j in range(len(self.source_codes_df))}
                scores.append({file:score for file,score in sorted(sourceCodeScores.items(),key=lambda tup: tup[1],reverse=True)})

            except Exception as e:
                logging.error(traceback.format_exc())
                scores.append({})                 
                
        self.all_bugs_df.loc[:,'total_score']=scores

    def evaluate(self):
        self.result=evaluate(self.all_bugs_df,self.source_codes_df)
        print("Result/"+self.project+":\n\t",'*'*4," MAP: ",self.result[0],'*'*4,'\n\t','*'*4," MRR: ",self.result[1],'*'*4,"\n","-"*100)

    def to_csv(self):
        BugReports_path=os.path.join(self.resultPath,'BugReports') 
        SourceFiles_path=os.path.join(self.resultPath,'SourceFiles')
        if not os.path.exists(BugReports_path):
            os.makedirs(BugReports_path)
        if not os.path.exists(SourceFiles_path):
            os.makedirs(SourceFiles_path)
        result_Bug_file=os.path.join(BugReports_path,self.project+"_BugReports.csv")
        result_source_file=os.path.join(SourceFiles_path,self.project+"_SourceFiles.csv")
        self.all_bugs_df.to_csv(result_Bug_file)
        if len(self.all_bugs_df)<300:
            self.source_codes_df.to_csv(result_source_file)
        
    def write_result(self):
        group_result=open(os.path.join(self.resultPath,"results_{}.csv".format(self.project)),'w')
        group_result.write("project , MAP , MRR , #ofBugReports\n")
        group_result.write(project+','+str(self.result[0])+','+str(self.result[1])+','+str(self.result[2])+"\n")
        group_result.close()
        

### MAIN

In [ ]:
def folder_structure(run_name):
    result_path=os.path.join(os.getcwd(),"Result",run_name,strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    return result_path
    
if __name__=="__main__":

    run_name='5-Localtfidf'
    result_path=folder_structure(run_name)
    BugLocalizer.dataFolder=os.path.join(os.getcwd(),'Data')
    if BugLocalizer.all_projects_bugreports.empty or BugLocalizer.all_projects_source_codes.empty:
        BugLocalizer.loadEverything()
    
    all_projects=set(BugLocalizer.all_projects_bugreports.project)

    for project in all_projects:
        core=BugLocalizer(project=project,result_path=result_path)
        core.execute()
            


<center><h1>Result</h1></center> 

In [ ]:

method='5-Localtfidf'
runNumber="run1"
all_results_csv=[os.path.join(os.getcwd(),"Result",method,runNumber,folder) 
                 for folder in listdir(os.path.join(os.getcwd(),"Result",method,runNumber)) if '.csv' in folder]
results_df=pd.DataFrame([])
for result_csv in all_results_csv:
    res=pd.read_csv(result_csv,index_col=[0],header=0)
    results_df=results_df.append(res)

project_size_df=pd.read_csv('project_size.csv',index_col=[0],header=0)
results_df=pd.merge(results_df, project_size_df,
                                      left_index=True,
                                      right_index=True)
results_df=results_df.reset_index()
results_df=results_df.set_index(' #ofBugReports')

results_df.to_csv(os.path.join(os.getcwd(),"Result",method,'result.csv'))
